<a href="https://colab.research.google.com/github/rakshitakedia-lab/Option-Pricing-Dashboard/blob/main/Rule_Based_Loan_Default_Prediction_%26_Credit_Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Simple Loan Default Prediction & Credit Scoring
================================================
Clean, fast credit risk assessment without training overhead

No installation needed - uses rule-based scoring only
"""

import pandas as pd
import numpy as np

class SimpleCreditScorer:
    """Rule-based credit scoring system"""

    def calculate_credit_score(self, applicant):
        """Calculate credit score based on financial rules"""
        score = 500  # Base score

        # CIBIL Score Impact (±200 points)
        cibil = applicant['cibil_score']
        if cibil >= 750:
            score += 150
        elif cibil >= 700:
            score += 100
        elif cibil >= 650:
            score += 50
        elif cibil >= 600:
            score += 0
        else:
            score -= 100

        # Income Level (±80 points)
        income = applicant['annual_income']
        if income >= 1000000:
            score += 80
        elif income >= 600000:
            score += 50
        elif income >= 400000:
            score += 20
        elif income < 250000:
            score -= 40

        # DTI Ratio (±100 points)
        dti = applicant['dti_ratio']
        if dti < 0.20:
            score += 60
        elif dti < 0.35:
            score += 20
        elif dti < 0.50:
            score -= 30
        else:
            score -= 100

        # Credit Utilization (±60 points)
        util = applicant['credit_utilization']
        if util < 0.30:
            score += 40
        elif util < 0.50:
            score += 10
        elif util < 0.75:
            score -= 20
        else:
            score -= 60

        # Employment Stability (±50 points)
        emp_length = applicant['employment_length']
        if emp_length >= 10:
            score += 50
        elif emp_length >= 5:
            score += 30
        elif emp_length >= 2:
            score += 10
        elif emp_length >= 1:
            score += 0
        else:
            score -= 20

        # Past Delinquencies (−80 points each)
        score -= applicant['num_delinquencies'] * 80

        # Recent Inquiries (−20 points each if >3)
        if applicant['num_inquiries_6m'] > 3:
            score -= (applicant['num_inquiries_6m'] - 3) * 20

        # LTV Ratio (±40 points)
        ltv = applicant['ltv_ratio']
        if ltv < 0.60:
            score += 40
        elif ltv < 0.80:
            score += 10
        elif ltv > 0.90:
            score -= 40

        # Age Factor (±30 points)
        age = applicant['age']
        if 30 <= age <= 50:
            score += 30
        elif 25 <= age < 30:
            score += 10
        elif age < 22:
            score -= 20

        # Normalize to 300-850 range
        score = max(300, min(850, score))

        return int(score)

    def calculate_pd(self, credit_score):
        """Convert credit score to Probability of Default"""
        # Sigmoid-based PD calculation
        # Higher score = Lower PD
        pd = 1 / (1 + np.exp((credit_score - 550) / 50))
        return min(0.99, max(0.01, pd))

    def get_risk_category(self, credit_score, pd):
        """Determine risk category"""
        if credit_score >= 750:
            return "Low Risk", "#28a745", "✅ APPROVE"
        elif credit_score >= 650:
            return "Medium Risk", "#ffc107", "⚠️ REVIEW"
        else:
            return "High Risk", "#dc3545", "❌ REJECT"

    def get_recommendation(self, credit_score, pd, applicant):
        """Provide detailed recommendation"""
        risk_cat, color, decision = self.get_risk_category(credit_score, pd)

        if credit_score >= 750:
            return decision, "Excellent creditworthiness. Approve with standard interest rate."
        elif credit_score >= 700:
            return decision, "Good credit profile. Approve with competitive terms."
        elif credit_score >= 650:
            return decision, "Acceptable risk. Consider slightly higher interest rate or additional documentation."
        elif credit_score >= 600:
            return decision, "Elevated risk. Requires detailed review. May need co-applicant or collateral."
        else:
            return decision, "High risk profile. Recommend rejection unless strong mitigating factors exist."


def get_user_input():
    """Get loan applicant details"""
    print("="*70)
    print("           LOAN APPLICATION ASSESSMENT")
    print("="*70)
    print("\nEnter borrower details:")
    print("-"*70)

    try:
        # Demographics
        age = int(input("Age [21-65]: ") or 35)

        # Employment & Income
        annual_income = float(input("Annual Income (₹): ") or 600000)
        employment_length = int(input("Employment Length (years): ") or 5)

        # Loan Details
        loan_amount = float(input("Loan Amount (₹): ") or 500000)
        property_value = float(input("Property Value (₹): ") or 1000000)

        # Credit Profile
        cibil_score = int(input("CIBIL Score [300-900]: ") or 750)
        num_delinquencies = int(input("Number of Past Delinquencies: ") or 0)
        num_inquiries_6m = int(input("Credit Inquiries (last 6 months): ") or 1)

        # Obligations
        existing_emi = float(input("Existing Monthly EMI (₹): ") or 10000)
        total_credit_limit = float(input("Total Credit Card Limit (₹): ") or 200000)
        credit_balance = float(input("Current Credit Card Balance (₹): ") or 50000)

        # Calculate derived metrics
        monthly_income = annual_income / 12
        dti_ratio = existing_emi / monthly_income
        ltv_ratio = loan_amount / property_value
        credit_utilization = credit_balance / (total_credit_limit + 1)

        applicant = {
            'age': age,
            'annual_income': annual_income,
            'employment_length': employment_length,
            'loan_amount': loan_amount,
            'property_value': property_value,
            'cibil_score': cibil_score,
            'num_delinquencies': num_delinquencies,
            'num_inquiries_6m': num_inquiries_6m,
            'existing_emi': existing_emi,
            'total_credit_limit': total_credit_limit,
            'credit_balance': credit_balance,
            'dti_ratio': dti_ratio,
            'ltv_ratio': ltv_ratio,
            'credit_utilization': credit_utilization
        }

        return applicant

    except ValueError as e:
        print(f"\nError: Invalid input - {e}")
        return None


def display_results(applicant, credit_score, pd, decision, recommendation):
    """Display assessment results"""
    print("\n" + "="*70)
    print("           CREDIT RISK ASSESSMENT RESULTS")
    print("="*70)


    # Key Metrics
    print(f"\n{'CREDIT SCORE:':<30} {credit_score}/850")
    print(f"{'PROBABILITY OF DEFAULT:':<30} {pd*100:.2f}%")

    risk_cat, color, dec = SimpleCreditScorer().get_risk_category(credit_score, pd)
    print(f"{'RISK CATEGORY:':<30} {risk_cat}")

    # Financial Ratios
    print("\n" + "-"*70)
    print("FINANCIAL METRICS")
    print("-"*70)
    print(f"{'Debt-to-Income (DTI) Ratio:':<30} {applicant['dti_ratio']:.2%}")
    print(f"{'Loan-to-Value (LTV) Ratio:':<30} {applicant['ltv_ratio']:.2%}")
    print(f"{'Credit Utilization:':<30} {applicant['credit_utilization']:.2%}")

    # Score Breakdown
    print("\n" + "-"*70)
    print("CREDIT PROFILE SUMMARY")
    print("-"*70)
    print(f"{'CIBIL Score:':<30} {applicant['cibil_score']}")
    print(f"{'Annual Income:':<30} ₹{applicant['annual_income']:,.0f}")
    print(f"{'Employment Years:':<30} {applicant['employment_length']}")
    print(f"{'Past Delinquencies:':<30} {applicant['num_delinquencies']}")
    print(f"{'Recent Inquiries:':<30} {applicant['num_inquiries_6m']}")

    # Final Recommendation
    print("\n" + "="*70)
    print(f"DECISION: {decision}")
    print("="*70)
    print(f"\n{recommendation}")
    print("\n" + "="*70)

    # Risk Interpretation
    if credit_score >= 750:
        print("\n✅ INTERPRETATION: Strong creditworthiness with minimal default risk.")
        print("   → Approve loan with standard terms and competitive interest rates.")
    elif credit_score >= 650:
        print("\n⚠️  INTERPRETATION: Moderate credit profile with manageable risk.")
        print("   → Consider approval with standard terms or slightly higher rate.")
    else:
        print("\n❌ INTERPRETATION: Elevated default risk based on credit metrics.")
        print("   → Recommend detailed review or rejection unless mitigating factors exist.")


def main():
    """Main execution"""
    print("="*70)
    print("     BANK LOAN DEFAULT PREDICTION & CREDIT SCORING")
    print("="*70)

    scorer = SimpleCreditScorer()

    # Get applicant details
    applicant = get_user_input()

    if applicant:
        # Calculate scores
        credit_score = scorer.calculate_credit_score(applicant)
        pd = scorer.calculate_pd(credit_score)
        decision, recommendation = scorer.get_recommendation(credit_score, pd, applicant)

        # Display results
        display_results(applicant, credit_score, pd, decision, recommendation)

        # Ask for another assessment
        print("\n" + "="*70)
        again = input("Assess another applicant? (yes/no): ").strip().lower()
        if again in ['yes', 'y']:
            print("\n" * 2)
            main()
        else:
            print("\nThank you for using the Credit Assessment System!")
            print("="*70)


if __name__ == "__main__":
    main()